# Welcome to DFL-Colab!

It's a limited  demo of original DeepFaceLab.

Version 2.0

# Overview
*   Extractor works in full functionality.
*   Training work without preview.
*   Converter works in full functionality.
*   You can import/export workspace with your Google Drive.
*   Google Colab machine active for 12 hours, so don't forget to export workspace



# Step 1. Clone Github repository and install requirements

* Clone Github repository or pull updates
* Requirements install is automatically
* After requirements wil be install you need restart runtime (you will see button in output)

In [0]:
#@title Clone or pull DeepFaceLab from Github

Mode = "clone" #@param ["clone", "pull"]

if (Mode == "clone"):
  !git clone https://github.com/iperov/DeepFaceLab.git
else:
  %cd /content/DeepFaceLab
  !git pull

!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image

# Step 2. Upload workspace from Google Drive

*   You can set filename of your workspace archive (for default: "workspace.zip")
*   Inside archive you must have a dir "workspace"

In [0]:
#@title Set filename to upload from Drive { form-width: "30%", display-mode: "form" }
Filename = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Create command
cmd = "/content/drive/My\ Drive/"+Filename+" /content"


#Copy zip archive of your workspace from Drive
%cp $cmd
%cd "/content"
!unzip $Filename
cmd = "/content/"+Filename
!rm $cmd

# Step 3. Extract and sorting
* Extract frames for SRC or DST video. For default extension of file "*.mp4", but you can change it
* Denoise SRC or DST video. "Factor" param set intesity of denoising
*   Detect and align faces with one of detectors. (S3FD - recommended). If you need, you can get frames with debug landmarks.
*   Export workspace to Google Drive after extract and sort it manually (Last block of Notebook)


In [0]:
#@title Extract frames
Video = "data_dst.mp4" #@param ["data_src.mp4", "data_dst.mp4"] {allow-input: true}

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst.mp4":
  cmd+= " --input-file workspace/data_dst.mp4 --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.mp4 --output-dir workspace/data_src/"
  
!python $cmd

In [0]:
#@title Denoise frames
Data = "data_dst" #@param ["data_src", "data_dst"]
Factor = 5 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [0]:
#@title Detect faces
Data = "data_dst" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "MT"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
if Detector == "S3FD":
  detect_type = "s3fd"
elif Detector == "MT":
  detect_type = "mt"

folder = "workspace/"+Data
folder_align = folder+"/aligned"
debug_folder = folder_align+"/debug"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_align

if Debug:
  cmd+= " --debug-dir "+debug_folder

cmd+=" --detector "+detect_type
  
%cd "/content"
!python $cmd

In [0]:
#@title Sort aligned
Data = "data_dst" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["hist", "hist-dissim", "face-yaw", "face-pitch", "blur", "final"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

# Step 4. Train model

*   Choose your model type, but SAE is recommend for everyone
*   Set model options on output field
*   You can't get preview in output. To see preview manually download image from SAE_history.
* Also you can see last preview if stop training and run code "Show last preview"

In [0]:
#@title Training
Model = "SAE" #@param ["SAE", "H128", "LIAEF128", "DF"]

cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [0]:
#@title ##Show last preview{ vertical-output: true }
Model = "SAE" #@param ["SAE", "H128", "LIAEF128", "DF"]

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import glob
import os
from IPython.display import clear_output

list_of_files = glob.glob('/content/workspace/model/'+Model+'_history/*') # * means all if need specific format then *.csv
img_path = max(list_of_files, key=os.path.getctime)

clear_output()
imdata = img.imread(img_path)
plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(imdata)
plt.show()

# Step 5. Convert frames

In [0]:
#@title Convert
Model = "SAE" #@param ["SAE", "H128", "LIAEF128", "DF"]

cmd = "DeepFaceLab/main.py convert --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [0]:
#@title Upload converted frames to Drive (optional)
Archive_name = "workspace_merged.zip" #@param {type:"string"}

cmd_zip = "-r "+Archive_name+" workspace/data_dst/merged"
cmd = "/content/"+Archive_name+" /content/drive/My\ Drive/"

!zip $cmd_zip
!cp $cmd

In [0]:
#@title Get result video and copy to Drive 

!python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4
!cp /content/workspace/result.mp4 /content/drive/My\ Drive/

# Other functions

In [0]:
#@title Operations with workspace
Mode = "Delete history preview" #@param ["Zip and upload", "Delete models", "Delete history preview"]

if Mode == "Zip and upload":
  !zip -r workspace.zip workspace
  !cp  /content/workspace.zip /content/drive/My\ Drive/
elif Mode == "Delete models":
  !rm -R workspace/model/*
elif Mode == "Delete history preview":
  !rm workspace/model/SAE_history/*
  !rm workspace/model/H128_history/*
  !rm workspace/model/LIAEF128_history/*
  !rm workspace/model/DF_history/*